# 句子切割

In [1]:
def _cut(sentence):
    """
    将一段文本切分成多个句子
    :param sentence:
    :return:
    """
    new_sentence = []
    sen = []
    for i in sentence:
        if i in ['。', '！', '？', '?'] and len(sen) != 0:
            sen.append(i)
            new_sentence.append("".join(sen))
            sen = []
            continue
        sen.append(i)

    if len(new_sentence) <= 1: # 一句话超过max_seq_length且没有句号的，用","分割，再长的不考虑了。
        new_sentence = []
        sen = []
        for i in sentence:
            if i.split(' ')[0] in ['，', ','] and len(sen) != 0:
                sen.append(i)
                new_sentence.append("".join(sen))
                sen = []
                continue
            sen.append(i)
    if len(sen) > 0:  # 若最后一句话无结尾标点，则加入这句话
        new_sentence.append("".join(sen))
    return new_sentence



In [ ]:
def cut_test_set(text_list,len_treshold):
    cut_text_list = []
    cut_index_list = []
    for text in text_list:

        temp_cut_text_list = []
        text_agg = ''
        if len(text) < len_treshold:
            temp_cut_text_list.append(text)
        else:
            sentence_list = _cut(text)  # 一条数据被切分成多句话
            for sentence in sentence_list:
                if len(text_agg) + len(sentence) < len_treshold:
                    text_agg += sentence
                else:
                    temp_cut_text_list.append(text_agg)
                    text_agg = sentence
            temp_cut_text_list.append(text_agg)  # 加上最后一个句子

        cut_index_list.append(len(temp_cut_text_list))
        cut_text_list += temp_cut_text_list

    return cut_text_list, cut_index_list


# 数据处理

In [3]:
#设置样本长度
text_length = 250
def from_ann2dic(r_ann_path, r_txt_path, w_path, w_file):
    q_dic = {}
    with codecs.open(r_ann_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip("\n\r")
            line_arr = line.split('\t')
            entityinfo = line_arr[1]
            entityinfo = entityinfo.split(' ')
            cls = entityinfo[0]
            start_index = int(entityinfo[1])
            end_index = int(entityinfo[2])
            length = end_index - start_index
            for r in range(length):
                if r == 0:
                    q_dic[start_index] = ("B-%s" % cls)
                else:
                    q_dic[start_index + r] = ("I-%s" % cls)

    with codecs.open(r_txt_path, "r", encoding="utf-8") as f:
        content_str = f.read()
        
    
    cut_text_list, cut_index_list = cut_test_set([content_str],text_length)
    
    i = 0
    for idx, line in enumerate(cut_text_list):
        w_path_ = "%s/%s-%s-new.txt" % (w_path, w_file,idx)
        with codecs.open(w_path_, "w", encoding="utf-8") as w:
            for str_ in line:
                if str_ is " " or str_ == "" or str_ == "\n" or str_ == "\r":
                    pass
                else:
                    if i in q_dic:
                        tag = q_dic[i]
                    else:
                        tag = "O"  # 大写字母O
                    w.write('%s %s\n' % (str_, tag))
                i+=1
            w.write('%s\n' % "END O")            

        

# 划分训练集和验证集

In [4]:
import glob
import numpy as np
file_list = glob.glob('./round1_train/train/*.txt') 

In [5]:
len(file_list)

770

In [14]:
from sklearn.model_selection import train_test_split,KFold
import os

In [22]:
kf = KFold(n_splits=5, shuffle=True, random_state=520).split(file_list)

In [23]:
file_list = np.array(file_list)

In [24]:
for i, (train_fold, test_fold) in enumerate(kf):
    print(len(file_list[train_fold]),len(file_list[test_fold]))
    train_filelist = list(file_list[train_fold])
    val_filelist = list(file_list[test_fold])
    os.system(f"mkdir  ./round1_train/train_new_{i}/")
    os.system(f"mkdir  ./round1_train/val_new_{i}/")
    import os
    import codecs
    data_dir = './round1_train/train/'
    for file in train_filelist:
        if file.find(".ann") == -1 and file.find(".txt") == -1:
            continue
        file_name = file.split('/')[-1].split('.')[0]
        r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
        r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
        w_path = f'./round1_train/train_new_{i}/'
        w_file = file_name
        from_ann2dic(r_ann_path, r_txt_path, w_path,w_file)
    import os
    import codecs
    data_dir = './round1_train/train/'
    for file in val_filelist:
        if file.find(".ann") == -1 and file.find(".txt") == -1:
            continue
        file_name = file.split('/')[-1].split('.')[0]
        r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
        r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
        w_path = f'./round1_train/val_new_{i}/'
        w_file = file_name
        from_ann2dic(r_ann_path, r_txt_path, w_path,w_file)

    w_path = f"./round1_train/data/train_{i}.txt"
    for file in os.listdir(f'./round1_train/train_new_{i}/'):
        path = os.path.join(f'./round1_train/train_new_{i}/', file)
        if not file.endswith(".txt"):  
            continue
        q_list = []
        print("开始读取文件:%s" % file)
        with codecs.open(path, "r", encoding="utf-8") as f:
            line = f.readline()
            line = line.strip("\n\r")
            while line != "END O":
                q_list.append(line)
                line = f.readline()
                line = line.strip("\n\r")
        print("开始写入文本%s" % w_path)
        with codecs.open(w_path, "a", encoding="utf-8") as f:
            for item in q_list:
                if item.__contains__('\ufeff1'):
                    print("===============")
                f.write('%s\n' % item)
            f.write('\n')
        f.close()

    w_path = f"./round1_train/data/val_{i}.txt"
    for file in os.listdir(f"./round1_train/val_new_{i}/"):
        path = os.path.join(f"./round1_train/val_new_{i}/", file)
        if not file.endswith(".txt"):  
            continue
        q_list = []

        with codecs.open(path, "r", encoding="utf-8") as f:
            line = f.readline()
            line = line.strip("\n\r")
            while line != "END O":
                q_list.append(line)
                line = f.readline()
                line = line.strip("\n\r")

        with codecs.open(w_path, "a", encoding="utf-8") as f:
            for item in q_list:
                if item.__contains__('\ufeff1'):
                    print("===============")
                f.write('%s\n' % item)
            f.write('\n')
        f.close()

616 154
开始读取文件:305-0-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:306-0-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:306-1-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:308-0-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:308-1-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:309-0-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:309-1-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:309-2-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:31-0-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:31-1-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:31-2-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:31-3-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:310-0-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:310-1-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:311-0-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:314-0-new.txt
开始写入文本./round1_train/data/train_0.txt
开始读取文件:315-0-new.txt
开始写入文本./round1_train/data/train

In [9]:
!mkdir  ./round1_train/train_new/
!mkdir ./round1_train/val_new/

# 训练集处理

In [10]:
import os
import codecs
data_dir = './round1_train/train/'
for file in train_filelist:
    if file.find(".ann") == -1 and file.find(".txt") == -1:
        continue
    file_name = file.split('/')[-1].split('.')[0]
    r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
    r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
    w_path = f'./round1_train/train_new_{i}/'
    w_file = file_name
    from_ann2dic(r_ann_path, r_txt_path, w_path,w_file)


# 验证集处理

In [11]:
import os
import codecs
data_dir = './round1_train/train/'
for file in val_filelist:
    if file.find(".ann") == -1 and file.find(".txt") == -1:
        continue
    file_name = file.split('/')[-1].split('.')[0]
    r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
    r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
    w_path = './round1_train/val_new/'
    w_file = file_name
    from_ann2dic(r_ann_path, r_txt_path, w_path,w_file)


# 训练集合并

In [ ]:

w_path = "./round1_train/data/train.txt"
for file in os.listdir('./round1_train/train_new/'):
    path = os.path.join("./round1_train/train_new", file)
    if not file.endswith(".txt"):  
        continue
    q_list = []
    print("开始读取文件:%s" % file)
    with codecs.open(path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
        while line != "END\tO":
            q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    print("开始写入文本%s" % w_path)
    with codecs.open(w_path, "a", encoding="utf-8") as f:
        for item in q_list:
            if item.__contains__('\ufeff1'):
                print("===============")
            f.write('%s\n' % item)
        f.write('\n')
    f.close()

# 验证集合并

In [25]:

w_path = "./round1_train/data/val.txt"
for file in os.listdir('./round1_train/val_new/'):
    path = os.path.join("./round1_train/val_new", file)
    if not file.endswith(".txt"):  
        continue
    q_list = []

    with codecs.open(path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
        while line != "END O":
            q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    
    with codecs.open(w_path, "a", encoding="utf-8") as f:
        for item in q_list:
            if item.__contains__('\ufeff1'):
                print("===============")
            f.write('%s\n' % item)
        f.write('\n')
    f.close()

# 原始验证集拷贝

In [29]:
for file in val_filelist:
    file_name = file.split('/')[-1].split('.')[0]
    r_ann_path = os.path.join("./round1_train/train", "%s.ann" % file_name)
    os.system("cp %s %s"%(file,"./round1_train/val_data"))
    os.system("cp %s %s"%(r_ann_path,"./round1_train/val_data"))
    print(file)

./round1_train/train/120.txt
./round1_train/train/988.txt
./round1_train/train/504.txt
./round1_train/train/618.txt
./round1_train/train/968.txt
./round1_train/train/989.txt
./round1_train/train/719.txt
./round1_train/train/263.txt
./round1_train/train/964.txt
./round1_train/train/184.txt
./round1_train/train/519.txt
./round1_train/train/307.txt
./round1_train/train/234.txt
./round1_train/train/447.txt
./round1_train/train/677.txt
./round1_train/train/839.txt
./round1_train/train/116.txt
./round1_train/train/253.txt
./round1_train/train/654.txt
./round1_train/train/281.txt
./round1_train/train/59.txt
./round1_train/train/627.txt
./round1_train/train/8.txt
./round1_train/train/77.txt
./round1_train/train/948.txt
./round1_train/train/480.txt
./round1_train/train/877.txt
./round1_train/train/930.txt
./round1_train/train/415.txt
./round1_train/train/778.txt
./round1_train/train/437.txt
./round1_train/train/186.txt
./round1_train/train/283.txt
./round1_train/train/318.txt
./round1_train/tra